# Introduction

In [84]:
# The idea of this competition, is to predict sales for mnay product families for stores in ecuador.
#Train and test share: date, store_nbr, family(product_family), onpromotion (number of products of the family that were on promotion)
#Train also has the sales.
#Test is what we are trying to predict in the competition

#Stores contains store metadata, we might want to join that data into our train and test tables to improve predictions if they are not redundant with the store_nbr (aka if at least some of the stores share their values).
 #stores metadata
#type and cluster are not redundant. see stores_df[['type', 'cluster']].drop_duplicates()#.pivot(columns='cluster')

#Daily oil price, keep in mind oil price affects ecuador economical health.
#We probably want to join it into our dataframe, but keep in mind it seems to have some null values. We probably want to fill downwards so that it takes its value from the previous date but well have to see. We could also average between the next and previous day.


#While its not clear it seems to be the total transactions of the store on that day


#A table containing holiday events. 
#its worth noting that some events were transfered in date so we need to take into account hte actual date.
#Also, keep in mind that some seem to be regional and some local. This could mean that we need to take into account the region in considering them.
#Some holidays were of type bridge, meaning  they are extra days added to the same holiday.
#This are often compensated by making some not working days(ie saturday) working days. This are of type workday.

#Additonally to these tables wages on the public sector are paid on the 15th and last day of each month which could affect supermarket sales
#A great earthquiake struck ecuador on april 16 2016 which resulted in donations possibly affecting sales.


#TLDR we need to add many features to the train and test df.
#First lets make some column names easier to understand. Specially after joining.

In [16]:
import os
import json
with open('./.kaggle/kaggle.json') as credentials_file:
    credentials_dict = json.load(credentials_file)
    os.environ['KAGGLE_USERNAME'] = credentials_dict['username']
    os.environ['KAGGLE_KEY'] = credentials_dict['key']
    import kaggle
from zipfile import ZipFile
import pandas as pd
import tensorflow as tf
import keras
from tqdm import tqdm

In [8]:
def download_dataset():
    kaggle.api.competition_download_files(competition='store-sales-time-series-forecasting', path='./dataset', force=False, quiet=True)
    with ZipFile('./dataset/store-sales-time-series-forecasting.zip') as dataset_zip:
        dataset_zip.extractall('./dataset')

def get_raw_dfs() ->tuple:
    """Get dataframes containing all the info in the dataset."""
    train_df_inc = pd.read_csv('./dataset/train.csv', index_col='id')
    test_df_inc = pd.read_csv('./dataset/test.csv', index_col='id')
    stores_df = pd.read_csv('./dataset/stores.csv', index_col='store_nbr')
    oil_df = pd.read_csv('./dataset/oil.csv') 
    transactions_df = pd.read_csv('./dataset/transactions.csv')
    holiday_events_df = pd.read_csv('./dataset/holidays_events.csv')
    sample_submission_df = pd.read_csv('./dataset/sample_submission.csv')
    return train_df_inc, test_df_inc, stores_df, oil_df, transactions_df, holiday_events_df, sample_submission_df

def rename_raw_dfs_cols(train_df_inc, test_df_inc, stores_df, oil_df, transactions_df, holiday_events_df, sample_submission_df) ->tuple:
    """Rename the columns of the raw dataframes so they are more easily understandable"""
    train_df_inc = train_df_inc.rename(columns={'family':'product_family', 'onpromotion':'products_of_family_on_promotion'})
    test_df_inc = test_df_inc.rename(columns={'family':'product_family'})
    oil_df = oil_df.rename(columns={'dcoilwtico':'oil_price'})
    stores_df = stores_df.rename(columns={'transactions':'all_products_transactions', 'type':'store_type', 'cluster':'store_cluster', 'city':'store_city', 'state':'store_state'})
    holiday_events_df = holiday_events_df.rename(columns={'type':'day_type', 'locale':'holiday_locale_type', 'locale_name':'holiday_locale','description':'holiday_description', 'transferred':'holiday_transferred'})
    return train_df_inc, test_df_inc, stores_df, oil_df, transactions_df, holiday_events_df, sample_submission_df

def complete_features_dataset(features_df:pd.DataFrame, stores_df:pd.DataFrame, oil_df:pd.DataFrame, transactions_df:pd.DataFrame, holiday_events_df:pd.DataFrame)->pd.DataFrame:
    """Add relevant columns from the auxiliary dataframes into a features dataset, be it the train set or the test set."""
    full_features_df = features_df.merge(oil_df, on='date',how='left')
    full_features_df = full_features_df.merge(stores_df, on=['store_nbr'], how='left') #Need to also consider date. 'date', nvm
    full_features_df = full_features_df.merge(holiday_events_df, on='date')
    full_features_df = full_features_df.merge(transactions_df, on=['date', 'store_nbr'])
    return full_features_df

def reorder_features_dataset(features_df):
    """Reorder the columns in the feature dataframe so the table becomes easier to understand and inspect. Does not affect the rows."""
    return features_df # TODO Order to columns more logically

def one_hot_encode_features_dataset(features_df):
    """ """
    return features_df
    #TODO #return the dataset and the references

def normalize_numerical_features(features_df):
    """ """
    return features_df
    #TODO #return the dataset and the normalizer

def fill_missing_values(features_df):
    """ """


def prepare_features_df(inc_dataframe:pd.DataFrame, stores_df, oil_df, transactions_df, holiday_events_df):
    #TODO READ and rename columns of the dataset, and then process them here.
    complete_features_df = complete_features_dataset(inc_dataframe)
    ordered_features_df = reorder_features_dataset(complete_features_df)
    return ordered_features_df

def prepare_datasets():
    train_df_inc, test_df_inc, stores_df, oil_df, transactions_df, holiday_events_df, sample_submission_df = rename_raw_dfs_cols(get_raw_dfs())
    train_y_truth = train_df_inc.pop('sales')
    train_df, test_df = complete_features_dataset(train_df_inc), complete_features_dataset(test_df_inc)
    return train_df, test_df, train_y_truth, sample_dataset


train_df, test_df, train_y_truth, sample_dataset = prepare_datasets()